In [1]:
from pymongo.mongo_client import MongoClient
uri = "mongodb://localhost:27017/mlops"
# Create a new client and connect to the server
client = MongoClient(uri)

In [2]:
databases = client['mlops']

In [3]:
collection = databases['mongopack']

In [6]:
# Sample data
data = {'companyName': 'phyCS',
     'product': 'Affordable AI',
     'courseOffered': 'Machine Learning with Deployment'}

In [7]:
data

{'companyName': 'phyCS',
 'product': 'Affordable AI',
 'courseOffered': 'Machine Learning with Deployment'}

In [8]:

collection.insert_one(data)

InsertOneResult(ObjectId('675e7488ca027ec24d27470c'), acknowledged=True)

In [9]:
all_record=collection.find()

In [10]:
all_record

In [11]:
for record in all_record:
    print(record)

{'_id': ObjectId('675e7488ca027ec24d27470c'), 'companyName': 'phyCS', 'product': 'Affordable AI', 'courseOffered': 'Machine Learning with Deployment'}


In [12]:
collection.insert_many(
    [
        {

    "coursename":"genai",
    "instrutorname":"sunny",
    "modeofsession":"english"
},

{

    "coursename":"mlops",
    "instrutorname":"sunnysavita",
    "modeofsession":"english&hindi"
}
        
    ]
)

InsertManyResult([ObjectId('675e75cdca027ec24d27470d'), ObjectId('675e75cdca027ec24d27470e')], acknowledged=True)

In [13]:
import pandas as pd
import json

In [28]:
class mongodb_operation:

    def __init__(self,client_url: str, database_name: str, collection_name: str=None):
        self.client_url=client_url
        self.database_name=database_name
        self.collection_name=collection_name

    def create_client(self):
        client=MongoClient(self.client_url)
        return client


    def create_database(self):
        client=self.create_client()
        database=client[self.database_name]
        return database
        
    def create_collection(self,collection=None):
        database=self.create_database()
        collection=database[collection]
        return collection

    def insert_record(self,record:dict,collection_name:str):
        if type(record)==list:
            for data in record:
                if type(data)!= dict:
                    raise TypeError("record must be in the dict")
            collection=self.create_collection(collection_name)
            collection.insert_many(record)
        elif type(record)==dict:
            collection=self.create_collection(collection_name)
            collection.insert_one(record)

    def bulk_insert(self,datafile:str,collection_name:str=None):
        self.path=datafile

        if self.path.endswith('.csv'):
            data=pd.read_csv(self.path,encoding='utf-8')

        elif self.path.endswith('.xlsx'):
            data=pd.read_excel(self.path,encoding='utf-8')
            
        datajson=json.loads(data.to_json(orient='record'))
        collection=self.create_collection()
        collection.insert_many(datajson)

    def find_one(self,collection_name:str,query:dict=None):
        collection=self.create_collection(collection_name)
        return collection.find_one(query)
    

    def find_all(self,collection_name:str,query:dict=None):
        # collection=self.create_collection(collection_name)
        if query:
            return self.create_collection(collection_name).find(query)
        else:
            return self.create_collection(collection_name).find()
        
    def update(self,collection_name:str,query:dict,new_values:dict):
        collection=self.create_collection(collection_name)
        collection.update_one(query, new_values)

    def delete(self,collection_name:str,query:dict):
        collection=self.create_collection(collection_name)
        collection.delete_one(query)

    

In [29]:
client_url = "mongodb://localhost:27017"
database = 'mlops'
collection_name = 'mongopack'

In [30]:

mongo=mongodb_operation(client_url,database,collection_name)

In [21]:

mongo.insert_record({"name":"sunny","designation":"data scientist","salary":"10k"},collection_name)

In [22]:
mongo.insert_record([{"name":"sunny","designation":"data scientist","salary":"10k"},
{"name":"sudhanshu","designation":"sr. data scientist","salary":"150k"},
{"name":"krish","designation":"lead data scientist","salary":"70k"}
],collection_name)

In [31]:
find_data = mongo.find_all(collection_name)

In [32]:
find_data

In [35]:
next(iter(find_data))

{'_id': ObjectId('675e75cdca027ec24d27470e'),
 'coursename': 'mlops',
 'instrutorname': 'sunnysavita',
 'modeofsession': 'english&hindi'}

In [36]:
for data in find_data:
    print(data)

{'_id': ObjectId('675e7ce0ca027ec24d274712'), 'name': 'sunny', 'designation': 'data scientist', 'salary': '10k'}
{'_id': ObjectId('675e7cf5ca027ec24d274714'), 'name': 'sunny', 'designation': 'data scientist', 'salary': '10k'}
{'_id': ObjectId('675e7cf5ca027ec24d274715'), 'name': 'sudhanshu', 'designation': 'sr. data scientist', 'salary': '150k'}
{'_id': ObjectId('675e7cf5ca027ec24d274716'), 'name': 'krish', 'designation': 'lead data scientist', 'salary': '70k'}
